## GCP

In [1]:
# %pip list | grep google-cloud-aiplatform
# %pip list | grep google-api-core
# %pip install google-cloud-aiplatform==1.43.0
# %pip install google-api-core==2.17.1

In [2]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

import geopandas as gpd
from shapely.geometry import Point

import http.client
import json
import urllib.parse
from typing import Dict, Any
import pandas as pd
import folium

In [3]:
vertexai.init(project = "adsp-capstone-property-pilot", location = "us-central1")

## Model Setting - Gemini pro 1.0

In [4]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [5]:
# Test chat
prompt = "Hello."
print(get_chat_response(chat, prompt))

Hello! How can I help you today?


## Settings

In [6]:
with open('/home/jupyter/zillow_api.txt', 'r') as file:
    zillow_api = file.read()

In [7]:
import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

headers = {
	"X-RapidAPI-Key": zillow_api,
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

In [8]:
with open('/home/jupyter/yelp_api.txt', 'r') as file:
    yelp_api = file.read()

In [9]:
yelp_api_key = yelp_api

## Import functions

In [10]:
import os
# os.chdir('/home/jupyter/property_pilot/scripts')

# import generate_api_filter
# import fetch_property_info
# import chat_response
# import yelp_feature

In [11]:
%run /home/jupyter/property_pilot/scripts/chat_response.py

In [12]:
%run /home/jupyter/property_pilot/scripts/generate_api_filter.py

In [13]:
%run /home/jupyter/property_pilot/scripts/fetch_property_info.py

In [14]:
%run /home/jupyter/property_pilot/scripts/yelp_property.py

In [15]:
%run /home/jupyter/property_pilot/scripts/neighborhood_rag.py

In [16]:
%run /home/jupyter/property_pilot/scripts/yelp_advisor.py

## Import data

In [17]:
# For RAG

# read csv
vector_store = pd.read_csv('/home/jupyter/property_pilot/data/vector_store.csv')
# embedding to numpy array
vector_store['embeddings'] = vector_store['embeddings'].apply(lambda x: np.array(list(map(float, x.split(',')))))

## Import prompts

In [18]:
os.chdir('/home/jupyter/property_pilot/prompts')

# prompt_classifier
file_path = 'instruction_classifier.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_classifier = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}") 
    
# prompt_apifilter
file_path = 'generate_prompt_apifilter.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_apifilter = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_categories
file_path = 'yelp_categories.txt'
try:
    with open(file_path, 'r') as file:
        yelp_categories = file.read().splitlines()
        categories_string = ','.join(yelp_categories)
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# prompt_property(for final output)
file_path = 'generate_prompt_property.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_property = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# instruction_rag_neighborhood
file_path = 'instruction_rag_neighborhood.txt'
try:
    with open(file_path, 'r') as file:
        instruction_rag_neighborhood = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_advisor_instructions (for yelp local advisor)
file_path = 'yelp_advisor_instructions.txt'
try:
    with open(file_path, 'r') as file:
        yelp_advisor_instruction = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_output_instructions (for yelp local advisor)
file_path = 'yelp_output_instructions.txt'
try:
    with open(file_path, 'r') as file:
        yelp_output_instructions = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")

## Property Plot main function

In [19]:
def pp_chatbot(user_query):
    
    # User's quety classify
    prompt_classifier = generate_prompt_classifier(instruction_prompt_classifier, user_query)
    response_int = int(get_chat_response(chat, prompt_classifier))
    
    # if the user is looking for a property     
    if response_int == 1:

        # get prompt for api filter 
        prompt_apifilter = generate_prompt_apifilter(instruction_prompt_apifilter, user_query)

        # get api filter
        apifilter = get_chat_response(chat, prompt_apifilter)
        apifilter = extract_json_to_dict(apifilter)

        # get listings from zillow
        response = requests.get(url, headers = headers, params = apifilter)

        # get top 3 listings(sorted by newest)
        # Define fields to extract
        fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc", "longitude", "latitude"]
        top_properties = extract_properties(response, 3, fields)

        # get description of the properties
        top_properties = fetch_descriptions(top_properties)

        # get resoFacts(detail) of the properties
        keys_to_fetch_resoFacts = [
            'hasGarage', 'hasPetsAllowed', 'heating', 'cooling', 'flooring', 'appliances',
            'laundryFeatures', 'associationFee',
            'livingArea', 'taxAnnualAmount', 'parkingFeatures', 'stories'
        ]
        top_properties = fetch_resoFacts(top_properties, headers, keys_to_fetch_resoFacts)

        # get school info of the properties
        top_properties = fetch_schools(top_properties, headers)
        
        # get restaurant info from yelp api
        yelp_results = fetch_top_businesses_near_properties(yelp_api_key, top_properties, user_query, yelp_categories, chat)
        
        # extract restaurant info
        yelp_fields = ['business_name', 'categories_titles', 'rating', 'latitude', 'longitude', 'display_address', 'url']
        top_restaurants = extract_business_info(yelp_results, 2, yelp_fields)
        
        # merge top_restaurants with top_properties
        top_properties = merge_property_and_restaurant_info(top_properties, top_restaurants)

        # neighborhood boundaries shapefile
        neighborhood_boundaries_path = '/home/jupyter/property_pilot/data/neighborhood_boundaries/geo_export_825d7df4-a9cd-4cef-b3d7-2ec1adc30204.shp'
        # neighborhood description file
        neighborhood_info_path = '/home/jupyter/property_pilot/data/neighborhood_info/neighborhood_info_final.csv'

        # load the neighborhood boundaries and description file
        neighborhoods = load_neighborhood_boundaries(neighborhood_boundaries_path)
        neighborhoods_info = load_neighborhood_info(neighborhood_info_path)

        # get neighborhood details of the properties
        top_properties = fetch_neighborhood(top_properties, neighborhoods)
        top_properties = fetch_neighborhood_info(top_properties, neighborhoods_info)

        # create property map
        property_map = create_property_map(top_properties)
        
        # create property_restaurant map
        property_restaurant_map = create_property_restaurant_map(top_properties)

        # change format
        fields_desc = fields + ['description', 'resoFacts', 'schools', 'neighborhood', 'neighborhood_description', 'restaurant_1', 'restaurant_2']
        property_info = format_properties(top_properties, fields_desc)

        # prompt for final output from Gemini
        prompt_final = generate_prompt_property(instruction_prompt_property, user_query, property_info, str(fields))

        # get final response from Gemini
        print(get_chat_response(chat, prompt_final))
        display(property_restaurant_map)

        return top_properties

    elif response_int == 2:
        # print('The query is about restaurant. I will answer by using Yelp API Filter')
        yelp_prompt = generate_prompt_yelp_advisor(yelp_advisor_instruction, user_query, categories_string)
        
        # get yelp filter string
        yelp_filter_str = get_chat_response(chat, yelp_prompt)
        
        # get yelp filter in json format
        yelp_filter = extract_json_to_dict(yelp_filter_str)
        
        # get response from yelp api
        yelp_advisor_results = yelp_advisor(yelp_api_key, yelp_filter)
        
        # create yelp restaurant map
        restaurant_map = create_yelp_restaurant_map(yelp_advisor_results)
        
        # generate yelp output prompt
        yelp_output_prompt = final_output_yelp_advisor(yelp_output_instructions, user_query, yelp_advisor_results)
        
        # get final response from Gemini
        print(get_chat_response(chat, yelp_output_prompt))
        display(restaurant_map)
        
    elif response_int == 3:
        # print('The query is about neighborhood. I will answer by using RAG')
        context = get_context(question = user_query, vector_store=vector_store, num_docs=10)
        prompt_rag_neighborhood = generate_prompt_rag_neighborhood(instruction_rag_neighborhood, context, user_query)
        print(get_chat_response(chat, prompt_rag_neighborhood))
        # return context
    else:
        print(get_chat_response(chat, user_query))

## Use case 1
### property search - detail info

In [17]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "I am looking for a two-bedroom apartment in Chicago."
top_properties = pp_chatbot(user_query)

## Two-bedroom apartments in Chicago

Based on your request for a two-bedroom apartment in Chicago, here are a few options that might interest you:

**1. Single-family home in Burnside:**

- Address: 9349 S Woodlawn Ave, Chicago, IL 60619
- Price: $119,000
- Bedrooms: 2
- Bathrooms: 1
- Highlights: 
    - Freshly painted
    - Large room sizes 
    - Expansive lot and backyard 
    - Unfinished basement with expansion possibilities
    - Quiet neighborhood with convenient access to local amenities, schools, and parks 
- Link: https://www.zillow.com/homedetails/9349-S-Woodlawn-Ave-Chicago-IL-60619/4114945_zpid/
- Image: https://photos.zillowstatic.com/fp/145c32a679f93e6424b7d87c747cb950-p_e.jpg
- More details about the house and neighborhood can be provided upon request. 


**2. Condo in West Ridge:**

- Address: 6255 N Oakley Ave #G, Chicago, IL 60659
- Price: $179,900
- Bedrooms: 2
- Bathrooms: 2
- Highlights:
    - Stunning and rarely seen garden apartment
    - Complete renovation i

In [18]:
top_properties

[{'propertyType': 'SINGLE_FAMILY',
  'address': '9349 S Woodlawn Ave, Chicago, IL 60619',
  'price': 119000,
  'bedrooms': 2,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/9349-S-Woodlawn-Ave-Chicago-IL-60619/4114945_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/145c32a679f93e6424b7d87c747cb950-p_e.jpg',
  'longitude': -87.5945,
  'latitude': 41.724407,
  'description': "Welcome to your dream starter home or investment property! This charming 2 bedroom, 1 bathroom residence has been freshly painted and boasts large room sizes, an expansive lot, inviting yard and an unfinished basement, offering endless expansion possibilities.  As you step through the front door, you'll be greeted by a cozy and inviting living space, perfect for creating lasting memories with loved ones. The well-designed layout enhances the functionality of the home, providing a nice flow between the living and kitchen area. The layout creates a warm and welcoming atmosphere. The heart

In [19]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Could you tell me about the first property in detail?"
print(get_chat_response(chat, user_query))

ResponseValidationError: The model response did not completed successfully.
Finish reason: 3.
Finish message: .
Safety ratings: [category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.042802147567272186
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.08897849172353745
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: HIGH
blocked: true
probability_score: 0.8121234178543091
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.13706977665424347
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.0697949230670929
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.04360952600836754
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.13006427884101868
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0645354762673378
].
To protect the integrity of the chat session, the request and response were not added to chat history.
To skip the response validation, specify `model.start_chat(res

In [ ]:
user_query = "Which neighborhood is the first property located in? Tell me more about that neighborhood."
print(get_chat_response(chat, user_query))

In [20]:
user_query = "What are some good schools nearby the first property?"
print(get_chat_response(chat, user_query))

## Schools near the 9349 S Woodlawn Ave property

The 9349 S Woodlawn Ave property is located in the Burnside neighborhood of Chicago, which has several public schools in the vicinity. Here are two schools within walking distance of the property:

**1. Washington H Elementary School:**

- Public elementary school
- Located 0.3 miles from the property
- Rated 5 stars on Greatschools
- Serves grades PK-8
- Student-teacher ratio: Not available
- Website: https://www.greatschools.org/illinois/chicago/1293-Washington--H-Elementary-School/

**2. Harlan Community Academy High School:**

- Public high school
- Located 1.5 miles from the property
- Rated 1 star on Greatschools
- Serves grades 9-12
- Student-teacher ratio: Not available
- Website: https://www.greatschools.org/illinois/chicago/1145-Harlan-Community-Academy-High-School/

**Additional information:**

- The "Resofacts" section of the property description provides additional information about the schools, including their parking feat

In [21]:
user_query = "I am interested in the first property. Should I buy it or rent it ?"
print(get_chat_response(chat, user_query))

## Making a decision on the first property: buy or rent?

Whether to buy or rent the first property (9349 S Woodlawn Ave) depends on a variety of factors specific to your situation and financial goals. Here's a breakdown of some key considerations to help you make an informed decision:

**Reasons to buy:**

* **Building equity:** Over time, as you pay down your mortgage, you'll gain equity in the property. This means you'll have a growing asset that you can eventually sell or use as leverage for future investments.
* **Potential for appreciation:** The value of real estate can increase over time, especially in desirable locations. If the property appreciates in value, you could potentially sell it for a profit.
* **Tax benefits:** Homeowners can deduct mortgage interest and property taxes from their federal income taxes, which can lead to significant tax savings.
* **Stability and control:** Owning a home gives you a sense of stability and control over your living space. You can make c

In [22]:
user_query = "What restaurants are nearby the first property? Please provide me with the yelp link"
print(get_chat_response(chat, user_query))

## Restaurants near the 9349 S Woodlawn Ave property

Here are two restaurants located near the 9349 S Woodlawn Ave property, along with their Yelp links:

**1. Wingz It Iz (New American, Fish & Chips)**

* Rating: 4.0 stars on Yelp
* Distance from property: 0.5 miles
* Yelp link: https://www.yelp.com/biz/wingz-it-iz-chicago?adjust_creative=6Rl6qXMI6EN3VkijeNTf0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=6Rl6qXMI6EN3VkijeNTf0A

**2. Flames Fish and Chicken (Burgers, Chicken Wings, Sandwiches)**

* Rating: 5.0 stars on Yelp
* Distance from property: 1.2 miles
* Yelp link: https://www.yelp.com/biz/flames-fish-and-chicken-chicago?adjust_creative=6Rl6qXMI6EN3VkijeNTf0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=6Rl6qXMI6EN3VkijeNTf0A

**Additional resources:**

* You can use Yelp to search for other restaurants in the area, filter by cuisine, price range, and other criteria.
* The "Resofacts" section of the property description also in

In [23]:
top_properties

[{'propertyType': 'SINGLE_FAMILY',
  'address': '9349 S Woodlawn Ave, Chicago, IL 60619',
  'price': 119000,
  'bedrooms': 2,
  'bathrooms': 1,
  'detailUrl': 'https://www.zillow.com/homedetails/9349-S-Woodlawn-Ave-Chicago-IL-60619/4114945_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/145c32a679f93e6424b7d87c747cb950-p_e.jpg',
  'longitude': -87.5945,
  'latitude': 41.724407,
  'description': "Welcome to your dream starter home or investment property! This charming 2 bedroom, 1 bathroom residence has been freshly painted and boasts large room sizes, an expansive lot, inviting yard and an unfinished basement, offering endless expansion possibilities.  As you step through the front door, you'll be greeted by a cozy and inviting living space, perfect for creating lasting memories with loved ones. The well-designed layout enhances the functionality of the home, providing a nice flow between the living and kitchen area. The layout creates a warm and welcoming atmosphere. The heart

## Use case 2
### Initial query: Restaurant info

In [20]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "Is there good Japanese food on the north side of chicago?"
pp_chatbot(user_query)

## Absolutely! There are many fantastic Japanese restaurants on the north side of Chicago, each with its own unique charm and delicious offerings. To help you narrow down your search, here are a few exceptional options, based on your preference:

**1. Gaijin:** 

* Located in the heart of River West, this modern Japanese restaurant features a vibrant atmosphere and a creative menu. They offer a wide variety of dishes, including sushi, ramen, and inventive small plates. Their **high ratings, extensive menu, and trendy ambiance** make them a great choice for a diverse and enjoyable dining experience.
* More information: https://www.yelp.com/biz/gaijin-chicago?adjust_creative=6Rl6qXMI6EN3VkijeNTf0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=6Rl6qXMI6EN3VkijeNTf0A

**2. Miru:**

* Situated in the sleek and modern Langham Hotel, Miru offers a more upscale experience. Their menu focuses on high-quality sushi and other Japanese dishes, with stunning plating and an e

In [21]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Which of these restaurants are budget-friendly?"
print(get_chat_response(chat, user_query))

## Budget-Friendly Options:

Out of the five restaurants I previously recommended, the two most budget-friendly options would be:

1. **RAMEN-SAN**: This lively spot in River North specializes in ramen, offering a variety of styles and flavors at accessible prices. Their ramen bowls typically range from $12-$16, making them a great value for a hearty and flavorful meal. 

2. **SUSHI-SAN**: This popular spot in River North is known for its excellent sushi at reasonable prices. They offer a wide range of sushi options, including classic rolls starting around $5 and specialty rolls around $10-$15. They also have a variety of other Japanese dishes available at similar price points.

**Here's a quick comparison to help you decide:**

| Restaurant        | Price Range (Sushi) | Price Range (Other Dishes) |
|-------------------|--------------------|------------------------------|
| RAMEN-SAN         | $12-$16            | $5-$12                       |
| SUSHI-SAN         | $5-$15            

In [23]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Could you provide me a link to Sushi-San's menu?"
print(get_chat_response(chat, user_query))

## Sushi-San Menu Links:

**Sushi-San Website Menu:** https://www.sushisanrestaurant.com/menu/

This link shows the full menu, including appetizers, sushi, main courses, and specialty rolls. It also has photos of some of the dishes.

**Sushi-San Yelp Menu:** https://www.yelp.com/menu/sushi-san-chicago

This link also shows the full menu, but it may be formatted differently. 

**Sushi-San Delivery Menu (DoorDash):** https://www.doordash.com/store/sushi-san-chicago-32642/

This link shows the menu specifically for delivery orders through DoorDash. 

**Please note that menus and prices can change, so it's always a good idea to check directly with the restaurant before you go.**

Is there anything else I can help you with? 



## Use case 3
### Initial query: Neighborhood info

In [51]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "What is the Hyde Park in Chicago like?"
pp_chatbot(user_query)

## Hyde Park: A Blend of History, Education, and Lush Greenery


Nestled on the South Side of Chicago, Hyde Park stands as a distinct blend of historic charm, academic prowess, and vibrant community spirit. Home to the renowned University of Chicago and breathtaking green spaces, Hyde Park presents a unique microcosm within the bustling metropolis, offering an enriching experience for residents and visitors alike.


### A Glimpse into Hyde Park's Character:

- **University Influence:** Hyde Park is inextricably intertwined with the University of Chicago. Its presence infuses the neighborhood with intellectual energy, a diverse student population, and access to world-class research and cultural events. The University's stunning campus architecture, including the iconic Rockefeller Memorial Chapel and Frank Lloyd Wright's Robie House, adds to the historical charm.

- **Cultural Hub:** The neighborhood celebrates a thriving arts scene with renowned institutions like the Museum of Science 

In [52]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Tell me more about the neighborhood?"
print(get_chat_response(chat, user_query))

## Unveiling Deeper Nuances of Hyde Park:

To provide an even more comprehensive understanding of Hyde Park, let's dive deeper into some specific aspects that define its unique character:

**1. Educational Excellence:**

- The University of Chicago: As the anchor of the neighborhood, the University of Chicago is a world-renowned institution attracting leading scholars, researchers, and students. Its presence contributes to a stimulating intellectual environment and offers access to renowned libraries, museums, and academic events.
- Other Educational Institutions: Hyde Park boasts several prestigious institutions beyond the University of Chicago. The renowned Laboratory Schools and the University of Chicago Charter School offer exceptional educational options for children.
- Lifelong Learning Opportunities: Hyde Park fosters a culture of continuous learning with resources like the Seminary Co-op Bookstore, hosting author events and discussions, and the Hyde Park Historical Society, pre